In [19]:
import os
from PIL import Image
import numpy as np
import torch
from torch.utils.data import DataLoader
from transformers import DPTImageProcessor, DPTForDepthEstimation

In [20]:
train_loader = DataLoader(CustomDepthDataset(), batch_size=8, shuffle=True)
model = DPTForDepthEstimation.from_pretrained("Intel/dpt-large", num_labels=1)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

for epoch in range(num_epochs):
    model.train()
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(**inputs)
        loss = torch.nn.functional.mse_loss(outputs.predicted_depth, labels)
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}, Loss: {loss.item()}")

# Save the fine-tuned model
model.save_pretrained('/content/DPT/')

NameError: name 'CustomDepthDataset' is not defined

In [14]:
def process_folder(input_folder, output_folder):
    processor = DPTImageProcessor.from_pretrained("Intel/dpt-large-ade")
    model = DPTForDepthEstimation.from_pretrained("Intel/dpt-large-ade")
    image_processor = AutoImageProcessor.from_pretrained("Intel/dpt-large-ade")
    model = DPTForSemanticSegmentation.from_pretrained("Intel/dpt-large-ade")

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for filename in os.listdir(input_folder):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tif', '.tiff')):
            base, ext = os.path.splitext(filename)
            input_path = os.path.join(input_folder, filename)
            output_path = os.path.join(output_folder, f"{base}_depth{ext}")

            image = Image.open(input_path).convert("RGB")

            inputs = processor(images=image, return_tensors="pt")

            with torch.no_grad():
                outputs = model(**inputs)
                predicted_depth = outputs.predicted_depth

            prediction = torch.nn.functional.interpolate(
                predicted_depth.unsqueeze(1),
                size=image.size[::-1],
                mode="bicubic",
                align_corners=False
            ).squeeze()

            output = prediction.cpu().numpy()
            formatted = (output * 255 / np.max(output)).astype("uint8")
            depth_image = Image.fromarray(formatted)
            depth_image.save(output_path)

            print(f"Processed {filename} and saved to {output_path}")


In [15]:
input_folder = '/content/drive/MyDrive/FaultLines/svi/40187_size300x800_fov120/'
output_folder = '/content/drive/MyDrive/FaultLines/svi/40187_size300x800_fov120/depth/'

In [16]:
process_folder(input_folder, output_folder)

Some weights of DPTForDepthEstimation were not initialized from the model checkpoint at Intel/dpt-large and are newly initialized: ['neck.fusion_stage.layers.0.residual_layer1.convolution1.bias', 'neck.fusion_stage.layers.0.residual_layer1.convolution1.weight', 'neck.fusion_stage.layers.0.residual_layer1.convolution2.bias', 'neck.fusion_stage.layers.0.residual_layer1.convolution2.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
from transformers import AutoImageProcessor, DPTForSemanticSegmentation
from PIL import Image
import requests

url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)

image_processor = AutoImageProcessor.from_pretrained("Intel/dpt-large-ade")
model = DPTForSemanticSegmentation.from_pretrained("Intel/dpt-large-ade")

inputs = image_processor(images=image, return_tensors="pt")

outputs = model(**inputs)
logits = outputs.logits

preprocessor_config.json:   0%|          | 0.00/285 [00:00<?, ?B/s]

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration. Please open a PR/issue to update `preprocessor_config.json` to use `image_processor_type` instead of `feature_extractor_type`. This warning will be removed in v4.40.


config.json:   0%|          | 0.00/6.91k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.37G [00:00<?, ?B/s]

Some weights of DPTForSemanticSegmentation were not initialized from the model checkpoint at Intel/dpt-large-ade and are newly initialized: ['neck.fusion_stage.layers.0.residual_layer1.batch_norm1.bias', 'neck.fusion_stage.layers.0.residual_layer1.batch_norm1.num_batches_tracked', 'neck.fusion_stage.layers.0.residual_layer1.batch_norm1.running_mean', 'neck.fusion_stage.layers.0.residual_layer1.batch_norm1.running_var', 'neck.fusion_stage.layers.0.residual_layer1.batch_norm1.weight', 'neck.fusion_stage.layers.0.residual_layer1.batch_norm2.bias', 'neck.fusion_stage.layers.0.residual_layer1.batch_norm2.num_batches_tracked', 'neck.fusion_stage.layers.0.residual_layer1.batch_norm2.running_mean', 'neck.fusion_stage.layers.0.residual_layer1.batch_norm2.running_var', 'neck.fusion_stage.layers.0.residual_layer1.batch_norm2.weight', 'neck.fusion_stage.layers.0.residual_layer1.convolution1.weight', 'neck.fusion_stage.layers.0.residual_layer1.convolution2.weight']
You should probably TRAIN this mo